INTRODUCTION


This notebook presents a comprehensive workflow for automating Excel-based data processing tasks using Python. The objective is to streamline repetitive operations such as reading data from multiple Excel files, extracting and transforming relevant information, and exporting the processed output in a structured format.

The workflow uses libraries such as pandas, openpyxl, and os to handle file manipulation, data extraction, and formatting. The modular design of the code ensures that it can be easily adapted or extended for similar data automation tasks in other contexts.



In [1]:
%pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.5-cp312-cp312-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_10_9_x86_64.whl (12.5 MB)
Using cached numpy-2.2.5-cp312-cp312-macosx_14_0_x86_64.whl (6.7 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install openpyxl


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyyaml

  Using cached PyYAML-6.0.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp312-cp312-macosx_10_9_x86_64.whl (183 kB)
Note: you may need to restart the kernel to use updated packages.


Modules Used:


pandas: For handling tabular data (Excel files).

os: For file path and environment access.

smtplib, EmailMessage: For constructing and sending emails.

unicodedata: For normalizing text input.

getpass: Secure password input (hidden input).

yaml: Load structured configuration data.

In [5]:
import pandas as pd
import os
import smtplib
from email.message import EmailMessage
import unicodedata
import getpass
import yaml  


These makes your DataFrame:

Show all rows,

Limit to 15 columns,

Display in a wider format,

Show up to 100 characters per column cell.



In [6]:
# Configure display options

#Displays all rows of the DataFrame
pd.set_option('display.max_rows', None)
#Limits display to a maximum of 15 columns
pd.set_option('display.max_columns', 15)
#Sets the total width of the display in characters to 200
pd.set_option('display.width', 200)
#Sets the maximum number of characters shown per column value to 100
pd.set_option('display.max_colwidth', 100)


The goal is to clean and normalize the input, especially if it's user-provided or copy-pasted from external sources
This function will help in normalizing the input by removing unwanted characters and spaces

This function:

Normalizes weird Unicode characters,

Replaces invisible non-breaking spaces,

Trims outer spaces,

→ making input clean, consistent, and safe to use in filters, comparisons, or keys.



In [7]:
#unicodedata.normalize("NFKD", s) This converts characters to their canonical decomposed form; "NFKD": Normalization Form KD – Compatibility Decomposition, which also removes some formatting.
#\xa0 is a non-breaking space (often copied from web pages or PDFs); Replaces it with a regular space ' ' to avoid formatting issues.
#strip() removes leading and trailing whitespace from the string

def clean_input(s):
    return unicodedata.normalize("NFKD", s).replace('\xa0', ' ').strip()


This function is a robust and format-aware loader that:

Automatically detects file type,

Reads it using the correct Pandas function,

Ensures only supported formats (.csv, .xls, .xlsx) are accepted,

Fails clearly for anything else.



In [8]:

def read_file(file_path):
    #splits the filename into two parts: ("filename", ".extension"),[1] gets the extension part, .lower() makes it lowercase
    ext = os.path.splitext(file_path)[1].lower()
    #If the file extension is .csv, use pandas.read_csv() to read it into a DataFrame.
    if ext == '.csv':
        return pd.read_csv(file_path)
    #If it's an Excel file (.xls for older format or .xlsx for newer), use pandas.read_excel() to read it
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path)
    #If it's neither .csv nor Excel, raise an error
    else:
        raise ValueError("Unsupported file format. Use .csv or .xlsx")

| Function     | Purpose                                                                |
| ------------ | ---------------------------------------------------------------------- |
| `clean_data` | Cleans column names and values, removes empty rows/columns             |
| `save_file`  | Saves the cleaned DataFrame to `.csv` or `.xlsx`, error if unsupported |


In [9]:

def clean_data(df):
    df.columns = df.columns.str.strip()
#Loops through all object-type columns (usually strings);Converts each value to a string (if not already).Strips leading/trailing spaces from all values in those columns.
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].astype(str).str.strip()
#Deletes rows where all values are NaN
    df.dropna(axis=0, how='all', inplace=True)
#Deletes columns where all values are NaN
    df.dropna(axis=1, how='all', inplace=True)
    return df

def save_file(df, output_path):
    #Extracts the file extension from the output path (like .csv or .xlsx) and makes it lowercase
    ext = os.path.splitext(output_path)[1].lower()
    #If the output format is .csv, saves using to_csv()
    if ext == '.csv':
        df.to_csv(output_path, index=False)
    #If the output format is .xlsx, saves using to_excel()
    elif ext in ['.xls', '.xlsx']:
        df.to_excel(output_path, index=False)
    else:
        raise ValueError("Unsupported output file format. Use .csv or .xlsx")


This function:

Builds an email with a subject, body, and file attachment,

Cleans the filename for compatibility,

Sends it securely using Gmail’s SMTP service.


Important Note: For Gmail, you need to:

Enable 2FA (Two-Factor Auth),

Use a Google App Password instead of your real password.

In [10]:
def send_email_with_attachment(sender, password, recipient, subject, body, attachment_path):
    # Creates a new EmailMessage object.Sets email headers: Subject, From, and To
    # Sets the email body text, explicitly using UTF-8 encoding to support all characters
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = recipient
    msg.set_content(body, charset='utf-8')
    #Opens the file in binary mode for reading.
    with open(attachment_path, 'rb') as f:
        #Reads the file content as raw bytes.
        file_data = f.read()
        filename = os.path.basename(attachment_path)

        #Removes accents and special Unicode characters, leaving a plain ASCII name.
        #Prevents filename issues in email clients or servers
        filename = unicodedata.normalize('NFKD', filename).encode('ascii', 'ignore').decode('ascii')
        msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=filename)
#Connects securely to Gmail's SMTP server over SSL (port 465)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        #Logs in using the sender's email credentials (you typically use an app password for Gmail)
        smtp.login(sender, password)
        #Sends the email message with the attachment.
        smtp.send_message(msg)


This function load_email_recipients() is designed to load a list of email addresses from either a YAML config file or (as a fallback) an Excel file

| Source           | Behavior                                                                 |
| ---------------- | ------------------------------------------------------------------------ |
| `email.yaml`     | Preferred. Extracts `emails` list from YAML file.                        |
| `email_ids.xlsx` | Fallback if YAML is missing. Extracts first column with "email" in name. |
| Error            | Prints error and returns an empty list.                                  |


In [11]:
def load_email_recipients():
    """Load email recipients from YAML configuration file or Excel file"""
    # Try to load from YAML first
    try:
        with open('email.yaml', 'r') as file:
            data = yaml.safe_load(file)
        return data.get('emails', [])
    except FileNotFoundError:
        # If YAML not found, try loading from Excel
        try:
            email_file = 'email_ids.xlsx'
            if os.path.exists(email_file):
                print(f"YAML file not found, loading emails from {email_file}")
                df = pd.read_excel(email_file)
                # Look for a column with 'email' in its name (case insensitive)
                email_cols = [col for col in df.columns if 'email' in col.lower()]
                if email_cols:
                    # Takes the first matching column, Removes NaN values with .dropna()
                    # Converts it into a list and returns it
                    email_list = df[email_cols[0]].dropna().tolist()
                    return email_list
                else:
                    print("No email column found in Excel file")
            return []
        except Exception as e:
            print(f"Error loading emails from Excel: {e}")
            return []
    except Exception as e:
        print(f"Error loading email configuration: {e}")
        return []


Main Logic (Data Filtering & Emailing)

This main() function orchestrates a complete data filtering and emailing workflow using Pandas and SMTP.

It:

Loads and cleans data,

Filters it based on user input,

Saves and optionally emails the result.

| Step           | Purpose                                       |
| -------------- | --------------------------------------------- |
| Load Data      | From CSV                                      |
| Clean Data     | Strip strings, drop empties                   |
| Preview        | Show data and columns                         |
| Filter         | User-specified column, operator, value        |
| Select Columns | Choose columns to retain                      |
| Save           | Save to Excel                                 |
| Email          | Optional, with YAML or Excel-based recipients |


In [14]:

def main():
    # Pre-filled file path
    file_path = "Mall_Customers.csv"

    try:
        df = read_file(file_path)
        df = clean_data(df)

        print("\n Full Data Preview (up to 15 columns):")
        print(df)

        print("\nColumns in file:")
        # Show only up to 15 columns
        print(", ".join(df.columns[:15]))  

        # Input filtering condition
        column = clean_input(input("\nEnter column name to filter by: "))
        operator = clean_input(input("Enter operator (==, !=, >, >=, <, <=): "))
        value = clean_input(input("Enter value to compare with: "))

        try:
            value = float(value)
        except ValueError:
            value = f'"{value}"'

        #Uses DataFrame.query() to filter the data accordingly.
        condition = f"`{column}` {operator} {value}"
        filtered_df = df.query(condition)

        # Column selection
        keep_columns_input = input("\nEnter columns to keep (comma-separated): ")
        keep_columns = [clean_input(col) for col in keep_columns_input.split(',') if clean_input(col) in filtered_df.columns]
        filtered_df = filtered_df[keep_columns]

        print("\n Filtered Data Preview:")
        print(filtered_df)

        # Save filtered result
        output_file = "filtered_output.xlsx"
        save_file(filtered_df, output_file)
        print(f"\n Filtered data saved to: {output_file}")

        # Email part
        #Asks if the user wants to email the output file
        send_choice = clean_input(input("Do you want to send the filtered file via email? (y/n): ").lower())
        if send_choice == 'y':
            sender = "dhruv.16940@sakec.ac.in"
            #Securely prompts for the app password using getpass
            password = clean_input(getpass.getpass("Enter your app password (hidden): "))
            
            # Load recipient emails from YAML file or Excel file
            available_emails = load_email_recipients()
            
            if available_emails:
                print("\nAvailable email recipients:")
                for i, email in enumerate(available_emails, 1):
                    print(f"{i}. {email}")
                
                selection = input("\nEnter recipient number or 'all' for all recipients: ")
                
                if selection.lower() == 'all':
                    recipient = ', '.join(available_emails)
                #If user enters a number, sends to that index
                #Defaults to the first email if input is invalid
                else:
                    try:
                        idx = int(selection) - 1
                        if 0 <= idx < len(available_emails):
                            recipient = available_emails[idx]
                        else:
                            print("Invalid selection, using first email")
                            recipient = available_emails[0]
                    except ValueError:
                        print("Invalid input, using first email")
                        recipient = available_emails[0]
            else:
                # Fallback to manual entry if no emails found in YAML or Excel
                recipient = clean_input(input("Enter recipient email address: "))
            #Defines subject/body
            #Sends the Excel file using the earlier-defined email function. 
           
            subject = "Filtered Data Report"
            body = "Please find attached the filtered data report."
            send_email_with_attachment(sender, password, recipient, subject, body, output_file)
            print(" Email sent successfully.")
    #Catches and prints any unexpected error in the entire process
    except Exception as e:
        print(f"\n Error: {e}")

if __name__ == "__main__":
    main()



 Full Data Preview (up to 15 columns):
     CustomerID   Genre  Age  Annual Income (k$)  Spending Score (1-100)
0             1    Male   19                  15                      39
1             2    Male   21                  15                      81
2             3  Female   20                  16                       6
3             4  Female   23                  16                      77
4             5  Female   31                  17                      40
5             6  Female   22                  17                      76
6             7  Female   35                  18                       6
7             8  Female   23                  18                      94
8             9    Male   64                  19                       3
9            10  Female   30                  19                      72
10           11    Male   67                  19                      14
11           12  Female   35                  19                      99
12         